In [ ]:
! pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from tqdm import tqdm

In [ ]:
# initialize Pose estimator
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
pose = mp_pose.Pose(
    model_complexity=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

In [ ]:
def stream_pose(model, video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        # read frame from capture object
        _, frame = cap.read()

        try:
            # convert the frame to RGB format
            RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # process the RGB frame to get the result
            results = model.process(RGB)

            print(results.pose_landmarks)
            # draw detected skeleton on the frame
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # show the final output
            cv2.imshow('Output', frame)
        except Exception  as e:
          print(e)
          break
        if cv2.waitKey(1) == ord('q'):
          break
    cap.release()
    cv2.destroyAllWindows()


def gen_skeleton(frame, connections, height, width,
                 mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
                 ):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles 
    for coord in frame:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add lines 
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3


def pose_estimator(model, path):
    # create capture object
    cap = cv2.VideoCapture(path)
    if (cap.isOpened() == False): 
        print("Error reading video file")
    
    # We need to set resolutions.
    # so, convert them from float to integer.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    size = (frame_width, frame_height)

    skeleton_video = []
    while cap.isOpened():
        # read frame from capture object
        _, frame = cap.read()

        try:
            # convert the frame to RGB format
            RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # process the RGB frame to get the result
            results = pose.process(RGB)
            try:
                frame_result = results.pose_landmarks.landmark
                point_list = []
                for idx, landmark in enumerate(frame_result):
                    # print(idx, landmark.x)
                    # print(type(landmark))
                    coord = [landmark.x, landmark.y, landmark.z]
                    point_list.append(coord)
                skeleton_video.append(point_list)
            except Exception as inst:
                print(inst)
                pass
            
        except Exception as inst:
            print(inst)
            break
        if cv2.waitKey(1) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

    return np.array(skeleton_video), frame_height, frame_width

def gen_video(points, save_file,file_id, frame_h, frame_w, 
                 mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)],
              temp_dir="temp",
              ):
    os.makedirs(f"{temp_dir}/{file_id}",exist_ok=True)
    #size = (frame_w, frame_h)
    #result = cv2.VideoWriter(save_file, 
    #                     cv2.VideoWriter_fourcc(*'MJPG'),
    #                     10, size)

    #for frame in [points[-110]]:
    for __id,frame in enumerate(points):
        __id = "0"*(6-len(str(__id))) + str(__id)
        skel_image = gen_skeleton(frame, mapping_list, frame_h, frame_w)
        skel_image = cv2.resize(skel_image,[int(frame_w*0.2),int(frame_h*0.2)])
        plt.imsave(f"{temp_dir}/{file_id}/{__id}.png",skel_image)

    os.system(f"ffmpeg -framerate 10 -pattern_type glob -i '/content/temp/{file_id}/*.png' -c:v libx264 -pix_fmt yuv420p '{save_file}'")
    shutil.rmtree(f"/content/temp/{file_id}")
        #result.write(skel_image[:,:,::-1])
    #result.release()

def save_arrays_file(skeletal_frames,video_size,location):
  print(skeletal_frames.shape)
  np.savez(location,coords=skeletal_frames,video_size=video_size)

In [ ]:
#shutil.rmtree("/content/temp")

In [ ]:
skel_res, fh, fw = pose_estimator(pose, '/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/video_dataset/sitting_down/a109-1310C.mp4')
gen_video(skel_res, 'sample_poes_gen.mp4',"sample_vid", fh, fw)
save_arrays_file(skel_res,[fh,fw],"bdv.npz")

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

(150, 33, 3)


In [ ]:
file_load = np.load("bdv.npz")
file_load.files

['coords', 'video_size']

In [ ]:
data_fol = "/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/HAR_Video/K10_V2"
path_parts = data_fol.split("/Datasets/")[1]

model_ident = "MediaPose"
set_of_classes = [x for x in os.listdir(data_fol)]
main_npz_directory = f"/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/skeleton_npz_outputs/{model_ident}/{path_parts}"
main_vid_directory = f"/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/skeleton_reconstruction_outputs/{model_ident}/{path_parts}"

In [ ]:
for class_type in set_of_classes:
  class_loc = os.path.join(data_fol,class_type)
  save_fol = os.path.join(main_vid_directory,class_type)
  save_numpy_fol = os.path.join(main_npz_directory,class_type)
  os.makedirs(save_fol,exist_ok=True)
  os.makedirs(save_numpy_fol,exist_ok=True)
  video_list = os.listdir(class_loc)

  if len(video_list)>300:
    video_list = video_list[:300]
  for vid in tqdm(video_list):
    video_id = vid.split(".")[0]
    if not os.path.exists(os.path.join(save_fol,f"{video_id}.mp4")):
      skel_res, fh, fw = pose_estimator(pose, os.path.join(class_loc,vid))
      if skel_res.shape[0] != 0:
        save_arrays_file(skel_res,[fh,fw],os.path.join(save_numpy_fol,f"{video_id}.npz"))
        gen_video(skel_res, os.path.join(save_fol,f"{video_id}.mp4"),video_id, fh, fw)
    else:
      print(f'{os.path.join(save_fol,f"{video_id}.mp4")} path exists !')

  0%|          | 0/10 [00:00<?, ?it/s]

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

(155, 33, 3)


 10%|█         | 1/10 [00:06<00:58,  6.51s/it]

'NoneType' object has no attribute 'landmark'
OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

(499, 33, 3)


 20%|██        | 2/10 [00:59<04:32, 34.11s/it]

...

 85%|████████▍ | 11/13 [01:29<00:16,  8.39s/it]

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

(180, 33, 3)


 92%|█████████▏| 12/13 [01:39<00:08,  8.75s/it]

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

(195, 33, 3)


100%|██████████| 13/13 [01:49<00:00,  8.41s/it]
0it [00:00, ?it/s]
